In [1]:
from cashe import get_cashed_input
from src.lstm_encoder import LSTMEmbedding
from src.data_local import MergedDataPreprocessing
import pandas as pd

In [2]:
df = get_cashed_input()

In [3]:
df

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,ELIGIBILITY_API_TRANS_ID,CLAIM_IDENTIFIER,ATTACHED_FILES,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END
0,9521,2024-04-04 22:57:57,230089,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,...,1680350.0,4430078.0,929362-34-4430078_OPDUCAF_2024-04-04_22-57-36_...,FILE,N,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31
1,9521,2024-04-04 23:02:21,230117,202.40,TSH,1.0,220.00,220.00,0.00,220.00,...,1643690.0,4430087.0,1083147-3-4430087_OPDUCAF_2024-04-04_23-01-52_...,FILE,N,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31
2,9521,2024-04-04 23:08:38,230160,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,...,1616370.0,4430098.0,1339075-6-4430098_OPDUCAF_2024-04-04_23-08-22_...,FILE,N,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31
3,9521,2024-04-04 22:26:08,229872,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,...,1638278.0,4430011.0,656928-60-4430011_OPDUCAF_2024-04-04_22-25-48_...,FILE,N,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31
4,9521,2024-04-04 22:34:22,229913,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,...,1657180.0,4430028.0,1019628-25-4430028_OPDUCAF_2024-04-04_22-33-56...,FILE,N,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,...,NaN,4404986.0,1348956-2-4404986_OPDUCAF_2024-01-23_17-17-31_...,FILE,N,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31
488829,10962,2024-01-28 01:01:05,204377,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,...,NaN,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,N,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31
488830,10962,2024-01-28 01:01:05,204377,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,...,NaN,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,N,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31
488831,10962,2024-01-28 01:01:05,204377,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,...,NaN,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,N,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31


In [13]:
#!pip install tf-keras --quiet
#!pip install transformers --quiet

In [5]:
preprocessing = MergedDataPreprocessing(df)

In [6]:
df_prep = preprocessing.columns_prep(service_encoding=True)

E:\Projects\Claims_Rejection\src\data_local.py:203: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:208: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


In [7]:
df_prep

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,CLAIM_IDENTIFIER,ATTACHED_FILES,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,PatientAgeRange
0,9521,2024-04-04 22:57:57,230089,56.12,1205,1.0,61.00,61.00,0.00,61.00,...,4430078.0,929362-34-4430078_OPDUCAF_2024-04-04_22-57-36_...,FILE,0,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31,6
1,9521,2024-04-04 23:02:21,230117,202.40,1165,1.0,220.00,220.00,0.00,220.00,...,4430087.0,1083147-3-4430087_OPDUCAF_2024-04-04_23-01-52_...,FILE,0,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31,5
2,9521,2024-04-04 23:08:38,230160,408.08,617,1.0,475.00,475.00,10.75,464.25,...,4430098.0,1339075-6-4430098_OPDUCAF_2024-04-04_23-08-22_...,FILE,0,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31,6
3,9521,2024-04-04 22:26:08,229872,24.27,396,1.0,25.55,25.55,4.85,20.70,...,4430011.0,656928-60-4430011_OPDUCAF_2024-04-04_22-25-48_...,FILE,0,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31,6
4,9521,2024-04-04 22:34:22,229913,80.64,917,1.0,90.00,90.00,14.40,75.60,...,4430028.0,1019628-25-4430028_OPDUCAF_2024-04-04_22-33-56...,FILE,0,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,930,1.0,324.39,324.39,0.00,324.39,...,4404986.0,1348956-2-4404986_OPDUCAF_2024-01-23_17-17-31_...,FILE,0,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31,8
488829,10962,2024-01-28 01:01:05,204377,1.06,770,1.0,1.14,1.14,0.00,1.14,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7
488830,10962,2024-01-28 01:01:05,204377,1.14,1206,5.0,1.77,1.77,0.00,1.77,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7
488831,10962,2024-01-28 01:01:05,204377,1.79,1058,1.0,1.93,1.93,0.00,1.93,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7


In [13]:
lstm_embedding = LSTMEmbedding()
df1 = df_prep.copy()
textual_col = ['SERVICE_DESCRIPTION', 'SERVICE_TYPE', 'OASIS_IOS_DESCRIPTION','PROVIDER_DEPARTMENT']
#df1

pre_embedd = []

for i in range(len(df1)):
    elem = ''
    for col in textual_col:
        elem = elem + str(df1[col].iloc[i]) + ' '
    elem = elem[:-1] + '.'
    pre_embedd.append(elem)
lstm_embedding.embedding_vector(concatenated_string,reload_model=True)

df_prep

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,CLAIM_IDENTIFIER,ATTACHED_FILES,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,PatientAgeRange
0,9521,2024-04-04 22:57:57,230089,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,...,4430078.0,929362-34-4430078_OPDUCAF_2024-04-04_22-57-36_...,FILE,0,H,2024-03-28 00:00:00,2024-04-11 00:00:00,2024-03-01,2024-03-31,6
1,9521,2024-04-04 23:02:21,230117,202.40,TSH,1.0,220.00,220.00,0.00,220.00,...,4430087.0,1083147-3-4430087_OPDUCAF_2024-04-04_23-01-52_...,FILE,0,H,2024-03-12 00:00:00,2024-03-26 00:00:00,2024-03-01,2024-03-31,5
2,9521,2024-04-04 23:08:38,230160,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,...,4430098.0,1339075-6-4430098_OPDUCAF_2024-04-04_23-08-22_...,FILE,0,H,2024-03-04 00:00:00,2024-03-18 00:00:00,2024-03-01,2024-03-31,6
3,9521,2024-04-04 22:26:08,229872,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,...,4430011.0,656928-60-4430011_OPDUCAF_2024-04-04_22-25-48_...,FILE,0,H,2024-03-10 00:00:00,2024-03-24 00:00:00,2024-03-01,2024-03-31,6
4,9521,2024-04-04 22:34:22,229913,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,...,4430028.0,1019628-25-4430028_OPDUCAF_2024-04-04_22-33-56...,FILE,0,H,2024-03-18 00:00:00,2024-04-01 00:00:00,2024-03-01,2024-03-31,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,9593,2024-01-23 17:20:54,204110,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,...,4404986.0,1348956-2-4404986_OPDUCAF_2024-01-23_17-17-31_...,FILE,0,IQ,2023-12-24 10:28:07,2023-12-28 12:07:18,2023-12-01,2023-12-31,8
488829,10962,2024-01-28 01:01:05,204377,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7
488830,10962,2024-01-28 01:01:05,204377,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7
488831,10962,2024-01-28 01:01:05,204377,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,...,4404989.0,1349595-2-4404989_LABCHRT1_2024-01-28_01-00-04...,FILE,0,IQ,2023-12-27 20:51:37,2023-12-31 17:16:53,2023-12-01,2023-12-31,7


In [8]:
from src.data_application import DataFrameProcessor

handling = DataFrameProcessor(df_prep)

In [9]:
df_prep.columns

Index(['CREATED_BY_USER', 'CREATION_DATE', 'VISIT_ID', 'NET_WITH_VAT',
       'SERVICE_DESCRIPTION', 'QTY', 'LINE_CLAIMED_AMOUNT',
       'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE',
       'LINE_ITEM_DISCOUNT', 'NET_AMOUNT', 'NET_VAT_AMOUNT',
       'PATIENT_VAT_AMOUNT', 'VAT_PERCENTAGE', 'TREATMENT_TYPE_INDICATOR',
       'SERVICE_TYPE', 'DURATION', 'QTY_STOCKED_UOM', 'UNIT_PRICE',
       'OASIS_IOS_DESCRIPTION', 'SEQUENCE_NO', 'UNIT_PRICE_STOCKED_UOM',
       'UNIT_PRICE_NET', 'DISCOUNT_PERCENTAGE', 'NOTES', 'OUTCOME',
       'APPROVED_QUNATITY', 'REQUEST_DATE', 'VISIT_NO', 'VISIT_DATE',
       'EMERGENCY_INDICATOR', 'PROVIDER_DEPARTMENT_CODE',
       'PROVIDER_DEPARTMENT', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE',
       'PATIENT_NO', 'EPISODE_NO', 'PATIENT_ID', 'MEMBERSHIP_NO',
       'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID', 'PATIENT_DOB',
       'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY',
       'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO'

In [10]:
## list of columns not needed in the modeling.
list_handler = ['PERIOD','DATE','_ID','_NO','_API','ATTACH','PATIENT_DOB','_USER','TREATMENT_SUB_TYPE','ELIGIBILITY_RESPONSE_SYSTEM', 'DOCTOR_LICENSE']

df_prep = handling._drop_columns_with_substring(list_handler)

In [11]:
df_prep

,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,...,PAYER_LICENSE,DEPARTMENT_TYPE,TREATMENT_TYPE,PURCHASER_CODE,STATUS,SUBMIT_CLAIM_OUTCOME,SUBMIT_CLAIM_MESSAGE,RES_STATUS,NEW_BORN,PatientAgeRange
0,56.12,1205,1.0,61.00,61.00,0.00,61.00,12.20,48.80,7.32,...,7001599658,18.0,1,2622,PARTIAL,PENDED,BE-1-1 Co-pay was not collected from memberBE-...,PARTIAL,0,6
1,202.40,1165,1.0,220.00,220.00,0.00,220.00,44.00,176.00,26.40,...,7001599658,6.0,1,2622,PARTIAL,PENDED,BE-1-7 Incorrect billing regimePartial Approved,PARTIAL,0,5
2,408.08,617,1.0,475.00,475.00,10.75,464.25,118.75,345.50,51.83,...,7001599658,11.0,1,2622,PARTIAL,PENDED,BE-1-7 Incorrect billing regimePartial Approved,PARTIAL,0,6
3,24.27,396,1.0,25.55,25.55,4.85,20.70,1.28,19.42,0.00,...,7001599658,18.0,1,2622,PARTIAL,PENDED,MN-1-1 Service is not clinically justified bas...,PARTIAL,0,6
4,80.64,917,1.0,90.00,90.00,14.40,75.60,18.00,57.60,8.64,...,7001599658,39.0,0,2622,PARTIAL,PENDED,BE-1-7 Incorrect billing regimeBE-1-7 Incorrec...,PARTIAL,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,242.48,930,1.0,324.39,324.39,0.00,324.39,113.54,210.85,31.63,...,7001571327,NaN,0,183,PARTIAL,PENDED,BE-1-4 Preauthorization is required and was no...,PARTIAL,0,8
488829,1.06,770,1.0,1.14,1.14,0.00,1.14,0.08,1.06,0.00,...,7001571327,NaN,0,183,PARTIAL,PENDED,MN-1-1 Service is not clinically justified bas...,PARTIAL,0,7
488830,1.14,1206,5.0,1.77,1.77,0.00,1.77,0.62,1.15,0.00,...,7001571327,NaN,0,183,PARTIAL,PENDED,MN-1-1 Service is not clinically justified bas...,PARTIAL,0,7
488831,1.79,1058,1.0,1.93,1.93,0.00,1.93,0.14,1.79,0.00,...,7001571327,NaN,0,183,PARTIAL,PENDED,MN-1-1 Service is not clinically justified bas...,PARTIAL,0,7


## New pipeline

In [1]:
from cashe import get_train_test_split,get_training_inputs, drop_nomodel_columns
from src.data_local import MergedDataPreprocessing
from src.lstm_encoder import LSTMEmbedding
from src.model_app import ModelApplication

In [2]:
df_train, df_test = get_train_test_split()

In [3]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train,df_test)

In [4]:
X_train = drop_nomodel_columns(X_train)
X_test = drop_nomodel_columns(X_test)
X_train

,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,...,TOTAL_NET_AMOUNT,TOTAL_DISCOUNT,TOTAL_DEDUCTIBLE,TOTAL_PATIENT_VATAMOUNT,TOTAL_NET_VAT_AMOUNT,DEPARTMENT_TYPE,TREATMENT_TYPE,PURCHASER_CODE,NEW_BORN,PatientAgeRange
0,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,12.20,48.80,7.32,...,1028.88,219.46,139.88,0.0,128.79,18.0,professional,2622,0,6
1,202.40,TSH,1.0,220.00,220.00,0.00,220.00,44.00,176.00,26.40,...,1029.75,320.25,100.00,0.0,154.46,6.0,professional,2622,0,5
2,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,118.75,345.50,51.83,...,1205.00,390.00,100.00,0.0,180.77,11.0,professional,2622,0,6
3,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,1.28,19.42,0.00,...,360.99,95.07,90.24,0.0,48.30,18.0,professional,2622,0,6
4,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,18.00,57.60,8.64,...,2765.79,291.19,103.74,0.0,412.62,39.0,institutional,2622,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,113.54,210.85,31.63,...,35058.44,5229.23,0.00,0.0,5231.53,NaN,institutional,183,0,8
488829,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,0.08,1.06,0.00,...,32309.30,13042.16,0.00,0.0,4608.10,NaN,institutional,183,0,7
488830,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,0.62,1.15,0.00,...,32309.30,13042.16,0.00,0.0,4608.10,NaN,institutional,183,0,7
488831,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,0.14,1.79,0.00,...,32309.30,13042.16,0.00,0.0,4608.10,NaN,institutional,183,0,7


In [5]:
preprocessing_train = MergedDataPreprocessing(X_train)
X_train_prep = preprocessing_train.columns_prep(service_encoding=False)

preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)

E:\Projects\Claims_Rejection\src\data_local.py:203: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[column] = self.df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:208: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['PatientAgeRange'] = self.df.PatientAgeRange.replace(age_encoding)


In [6]:
X_train_prep

,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,...,TOTAL_NET_AMOUNT,TOTAL_DISCOUNT,TOTAL_DEDUCTIBLE,TOTAL_PATIENT_VATAMOUNT,TOTAL_NET_VAT_AMOUNT,DEPARTMENT_TYPE,TREATMENT_TYPE,PURCHASER_CODE,NEW_BORN,PatientAgeRange
0,56.12,Unlisted Code,1.0,61.00,61.00,0.00,61.00,12.20,48.80,7.32,...,1028.88,219.46,139.88,0.0,128.79,18.0,1,2622,0,6
1,202.40,TSH,1.0,220.00,220.00,0.00,220.00,44.00,176.00,26.40,...,1029.75,320.25,100.00,0.0,154.46,6.0,1,2622,0,5
2,408.08,In vivo laboratory,1.0,475.00,475.00,10.75,464.25,118.75,345.50,51.83,...,1205.00,390.00,100.00,0.0,180.77,11.0,1,2622,0,6
3,24.27,ELICA-M CREAM 30 GM,1.0,25.55,25.55,4.85,20.70,1.28,19.42,0.00,...,360.99,95.07,90.24,0.0,48.30,18.0,1,2622,0,6
4,80.64,Peripheral intravenous cannula,1.0,90.00,90.00,14.40,75.60,18.00,57.60,8.64,...,2765.79,291.19,103.74,0.0,412.62,39.0,0,2622,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488828,242.48,Quantitative Troponin (Protein) Analysis,1.0,324.39,324.39,0.00,324.39,113.54,210.85,31.63,...,35058.44,5229.23,0.00,0.0,5231.53,NaN,0,183,0,8
488829,1.06,NEBILET 5MG TABLETS,1.0,1.14,1.14,0.00,1.14,0.08,1.06,0.00,...,32309.30,13042.16,0.00,0.0,4608.10,NaN,0,183,0,7
488830,1.14,Unlisted Procedure Code,5.0,1.77,1.77,0.00,1.77,0.62,1.15,0.00,...,32309.30,13042.16,0.00,0.0,4608.10,NaN,0,183,0,7
488831,1.79,SALURIN 20MG-2ML AMPOULES,1.0,1.93,1.93,0.00,1.93,0.14,1.79,0.00,...,32309.30,13042.16,0.00,0.0,4608.10,NaN,0,183,0,7


In [7]:
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep)
X_test_encoded = preprocessing_train.column_embedding(X_test_prep)

In [8]:
X_test_encoded

,NET_WITH_VAT,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT,PATIENT_VAT_AMOUNT,...,CombinedText7,CombinedText8,CombinedText9,CombinedText10,CombinedText11,CombinedText12,CombinedText13,CombinedText14,CombinedText15,CombinedText16
9,27.36,2.0,28.80,28.80,5.47,23.33,1.44,21.89,0.00,0.0,...,285,54,6,70,36,43,73,22,0,0
26,12.16,20.0,13.15,13.15,6.08,7.07,0.99,6.08,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
27,50.18,10.0,54.26,54.26,10.04,44.22,4.07,40.15,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
28,27.47,3.0,29.70,29.70,5.49,24.21,2.23,21.98,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
69,131.26,1.0,163.06,163.06,0.00,163.06,48.92,114.14,17.12,0.0,...,19,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488709,836.67,1.0,1053.50,1053.50,76.00,977.50,316.05,661.45,99.22,11.4,...,617,32,205,703,172,37,200,441,0,0
488710,134.40,1.0,120.00,120.00,24.00,96.00,0.00,96.00,14.40,3.6,...,0,0,0,0,0,0,0,0,0,0
488775,12.49,20.0,13.15,13.15,2.50,10.65,0.66,9.99,0.00,0.0,...,0,0,0,0,0,0,0,0,0,0
488776,12.67,1.0,13.70,13.70,2.53,11.17,1.03,10.14,0.00,0.0,...,68,67,45,6,2,35,2,68,0,0


In [9]:
model_runner = ModelApplication(X_train= X_train,y_train= y_train['OUTCOME'],X_test= X_test,y_test= y_test['OUTCOME'])

In [10]:
model_runner.get_model(force_retrain=True)

XGBoost Model is loading from disk..


In [11]:
model_runner.evaluate_model()

{'XGBoost': {'Accuracy': 0.75,
  'Precision': 0.74,
  'Recall': 0.79,
  'F1 Score': 0.76}}

In [19]:
y_train

,OUTCOME,SUBMIT_CLAIM_MESSAGE
0,0,BE-1-1 Co-pay was not collected from memberBE-...
1,0,BE-1-7 Incorrect billing regimePartial Approved
2,1,BE-1-7 Incorrect billing regimePartial Approved
3,1,MN-1-1 Service is not clinically justified bas...
4,0,BE-1-7 Incorrect billing regimeBE-1-7 Incorrec...
...,...,...
488828,0,BE-1-4 Preauthorization is required and was no...
488829,1,MN-1-1 Service is not clinically justified bas...
488830,1,MN-1-1 Service is not clinically justified bas...
488831,0,MN-1-1 Service is not clinically justified bas...


In [36]:
import re
from transformers import pipeline
import os
import joblib

class RejectionCategory:
    def __init__(self, model_path='data/bart_model/bart_zero_classifier'):
        if model_path and os.path.exists(model_path):
            self.classifier = joblib.load(model_path)
        else:
            self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

        self.categories = [
            "Medical Justification Denials",
            "Contractual or Financial Benefit Denials",
            "Information Errors or Omissions Denials"]
        print('Model loading is done')

    def save_classifier(self, model_path):
        joblib.dump(self.classifier, model_path)

    def classify_message(self, message,predictions):
        result = self.classifier(message, self.categories)
        labels = []
        for i in range(len(result)):
            if predictions[i] == 0:
                most_probable_label = result[i]['labels'][0]
                labels.append(most_probable_label)
            else:
                labels.append('APPROVED_PREDICTION')
            if i%10 == 0:
                print(f"A batch of {i} is done")
        return labels

    def preprocess_messages(self, messages,predictions):
        cleaned_messages = []
        nphies_code_pattern = r'[A-Z]{2}-\d+-\d+'

        for looper in range(len(messages)):
            if predictions[looper] == 0:
                message = messages[looper]
                message = str(message).strip().strip("'''")

                message = re.sub(f"({nphies_code_pattern})", r' \1 ', message)
                parts = message.split()

                unique_parts = []
                seen = set()
                for part in parts:
                    if part not in seen:
                        seen.add(part)
                        unique_parts.append(part)

                cleaned_message = ' '.join(unique_parts)
                cleaned_message = re.sub(r'([a-z])([A-Z])', r'\1 \2', cleaned_message)
                cleaned_messages.append(cleaned_message.strip())
            else:
                cleaned_messages.append('APPROVED_PREDICTION')

        return cleaned_messages

In [ ]:
#from src.rejection_clustering import RejectionCategory

rejection_category = RejectionCategory()

messages  = y_test['SUBMIT_CLAIM_MESSAGE'].tolist()
predicted_labels = y_test['OUTCOME'].tolist()

In [38]:
cleaned_messages = rejection_category.preprocess_messages(messages,predicted_labels)

In [ ]:
classification_result = rejection_category.classify_message(cleaned_messages,predicted_labels)